In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import chrome
from selenium import webdriver

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://twitter.com/login')

In [4]:
username = driver.find_element_by_xpath('//input[@name = "session[username_or_email]"]')
username.send_keys('error_402')

In [5]:
my_password = getpass()

 ··········


In [6]:
password = driver.find_element_by_xpath('//input[@name = "session[password]"]')
password.send_keys('my_password')

In [7]:
password.send_keys(Keys.RETURN)

In [8]:
search_input = driver.find_element_by_xpath('//input[@aria-label = "Search query"]')
search_input.send_keys('#polynotes')

In [9]:
search_input.send_keys(Keys.RETURN)

In [10]:
driver.find_element_by_link_text('Latest').click()

In [11]:
cards = driver.find_element_by_xpath('//div[@data-testid = "tweet"]')

In [12]:
card = cards

In [13]:
card.find_element_by_xpath('.//span').text

'Ozil'

In [14]:
#twitter handle
card.find_element_by_xpath('.//span[contains(text(),"@")]').text


'@fakemoneyplug'

In [15]:
#post date
card.find_element_by_xpath('.//time').get_attribute('datetime')

'2020-12-13T10:37:07.000Z'

In [16]:
#content of tweet
comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
responding = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
comment + responding

'Wondering what to do with your black notes? #ssdchemicalsolition made easy.\n\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#ssdchemicalsolution #blacknotes #makemoney #makemoneyonline #cashtips #entrepreneur #businesstips #polynotes #propmoneyonline #fakemoneyonline #MotivationWondering what to do with your black notes? #ssdchemicalsolition made easy.\n\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#ssdchemicalsolution #blacknotes #makemoney #makemoneyonline #cashtips #entrepreneur #businesstips #polynotes #propmoneyonline #fakemoneyonline #Motivation'

In [17]:
#reply count
card.find_element_by_xpath('.//div[@data-testid = "reply"]').text

'1'

In [18]:
#retweet
card.find_element_by_xpath('.//div[@data-testid = "retweet"]').text

''

In [19]:
#likes
card.find_element_by_xpath('.//div[@data-testid = "like"]').text

''

In [20]:
def get_tweet_data(card):
    """Extractdata from tweet data"""
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid = "reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid = "retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid = "like"]').text
        
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet
            

In [21]:
get_tweet_data(card)

('Ozil',
 '@fakemoneyplug',
 '2020-12-13T10:37:07.000Z',
 'Wondering what to do with your black notes? #ssdchemicalsolition made easy.\n\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#ssdchemicalsolution #blacknotes #makemoney #makemoneyonline #cashtips #entrepreneur #businesstips #polynotes #propmoneyonline #fakemoneyonline #MotivationWondering what to do with your black notes? #ssdchemicalsolition made easy.\n\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#ssdchemicalsolution #blacknotes #makemoney #makemoneyonline #cashtips #entrepreneur #businesstips #polynotes #propmoneyonline #fakemoneyonline #Motivation',
 '1',
 '',
 '')

In [ ]:
data =[]

In [38]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

InvalidSessionIdException: Message: invalid session id


In [37]:
with open('LL_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)